In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Aggregations").getOrCreate()

23/06/08 09:21:03 WARN Utils: Your hostname, FM-PC-LT-323 resolves to a loopback address: 127.0.1.1; using 172.16.5.219 instead (on interface wlp0s20f3)
23/06/08 09:21:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/08 09:21:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/08 09:21:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/06/08 09:21:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/06/08 09:21:05 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [6]:
retail_data_df = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("data/csv/retail_data/*.csv")\
    .coalesce(5)

retail_data_df.cache()
retail_data_df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   537226|    22811|SET OF 6 T-LIGHTS...|       6|2010-12-06 08:34:00|     2.95|   15987.0|United Kingdom|
|   537226|    21713|CITRONELLA CANDLE...|       8|2010-12-06 08:34:00|      2.1|   15987.0|United Kingdom|
|   537226|    22927|GREEN GIANT GARDE...|       2|2010-12-06 08:34:00|     5.95|   15987.0|United Kingdom|
|   537226|    20802|SMALL GLASS SUNDA...|       6|2010-12-06 08:34:00|     1.65|   15987.0|United Kingdom|
|   537226|    22052|VINTAGE CARAVAN G...|      25|2010-12-06 08:34:00|     0.42|   15987.0|United Kingdom|
|   537226|    22705|   WRAP GREEN PEARS |      25|2010-12-06 08:34:00|     0.42|   15987.0|United Kingdom|
|   537226|    20781|GOLD EA

In [9]:
retail_data_df.count()
# 14022

14022

In [12]:
retail_data_df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [ ]:
from pyspark.sql.functions import * 

In [20]:
from pyspark.sql.functions import count,countDistinct,approx_count_distinct
retail_data_df.select(count("StockCode")).show() # 14022

retail_data_df.select(countDistinct("StockCode")).show() # 2230

retail_data_df.select(approx_count_distinct("StockCode",0.2)).show() # 2155 

+----------------+
|count(StockCode)|
+----------------+
|           14022|
+----------------+

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     2230|
+-------------------------+

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            2155|
+--------------------------------+



In [22]:
from pyspark.sql.functions import first, last

retail_data_df.select(first("InvoiceDate"), last("InvoiceDate")).show()

+-------------------+-------------------+
| first(InvoiceDate)|  last(InvoiceDate)|
+-------------------+-------------------+
|2010-12-06 08:34:00|2010-12-03 17:28:00|
+-------------------+-------------------+



In [26]:
from pyspark.sql.functions import sum, count, avg, expr, min, max
retail_data_df\
    .select(count("Quantity").alias("total_transactions"),
            sum("Quantity").alias("total_purchases"),
            avg("Quantity").alias("avg_purchases"),
            expr("mean(Quantity)").alias("mean_purchases"))\
    .selectExpr("total_purchases/total_transactions","avg_purchases","mean_purchases")\
    .show()

retail_data_df\
    .select(min("Quantity"),max("Quantity"))\
    .show()



+--------------------------------------+------------------+------------------+
|(total_purchases / total_transactions)|     avg_purchases|    mean_purchases|
+--------------------------------------+------------------+------------------+
|                    7.1659535016402796|7.1659535016402796|7.1659535016402796|
+--------------------------------------+------------------+------------------+

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|        -9360|         2880|
+-------------+-------------+



In [29]:
from pyspark.sql.functions import sum, sumDistinct, sum_distinct

retail_data_df.select(sum("Quantity"),sumDistinct("Quantity"),sum_distinct("Quantity")).show()

+-------------+----------------------+----------------------+
|sum(Quantity)|sum(DISTINCT Quantity)|sum(DISTINCT Quantity)|
+-------------+----------------------+----------------------+
|       100481|                  4844|                  4844|
+-------------+----------------------+----------------------+



In [34]:
from pyspark.sql.functions import variance, stddev
from pyspark.sql.functions import var_pop, stddev_pop
from pyspark.sql.functions import var_samp, stddev_samp

retail_data_df\
    .select(variance("Quantity").alias("Default_var_samp(Quantity)"),
            stddev("Quantity").alias("Default_stddev_samp(Quantity)"),
            var_pop("Quantity"), 
            var_samp("Quantity"),
            stddev_pop("Quantity"), 
            stddev_samp("Quantity"))\
    .show()

+--------------------------+-----------------------------+-----------------+------------------+--------------------+---------------------+
|Default_var_samp(Quantity)|Default_stddev_samp(Quantity)|var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+--------------------------+-----------------------------+-----------------+------------------+--------------------+---------------------+
|          7843.58539520731|            88.56401862611763|7843.026018128776|  7843.58539520731|   88.56086053177654|    88.56401862611763|
+--------------------------+-----------------------------+-----------------+------------------+--------------------+---------------------+



In [35]:
from pyspark.sql.functions import skewness, kurtosis

retail_data_df.select(skewness("Quantity"), kurtosis("Quantity")).show()

+------------------+------------------+
|skewness(Quantity)|kurtosis(Quantity)|
+------------------+------------------+
|-80.58047369410305| 9024.825827028237|
+------------------+------------------+



In [37]:
from pyspark.sql.functions import corr, covar_pop, covar_samp

retail_data_df\
    .select(corr("InvoiceNo", "Quantity"), 
            covar_samp("InvoiceNo", "Quantity"),
            covar_pop("InvoiceNo", "Quantity"))\
    .show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|      -0.0488744918095809|             -623.2038275092896|            -623.1589054428649|
+-------------------------+-------------------------------+------------------------------+



In [39]:
from pyspark.sql.functions import collect_set, collect_list

retail_data_df.agg(collect_set("Country"), collect_list("Country")).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



In [40]:
retail_data_df.groupBy("InvoiceNo").count().show()
retail_data_df.groupBy("CustomerId").count().show()
retail_data_df.groupBy("InvoiceNo", "CustomerId").count().show()

+---------+-----+
|InvoiceNo|count|
+---------+-----+
|   537252|    1|
|   537439|    1|
|   537246|    7|
|   537256|   11|
|   537350|   12|
|   537228|    1|
|   537298|    9|
|   537436|    1|
|   537368|   15|
|   537245|    3|
|   537412|    1|
|   537243|    2|
|   537351|   32|
|   537354|   38|
|  C537373|    1|
|   537435|   14|
|  C537408|    2|
|   537378|   20|
|   537231|   18|
|   537312|    9|
+---------+-----+
only showing top 20 rows

+----------+-----+
|CustomerId|count|
+----------+-----+
|   13094.0|    3|
|   17884.0|   64|
|   16858.0|   10|
|   15898.0|   59|
|   16550.0|   51|
|   17320.0|   22|
|   18219.0|   13|
|   13089.0|   41|
|   17218.0|   51|
|   16654.0|    9|
|   14409.0|    3|
|   15899.0|    4|
|   15987.0|    9|
|      null| 4195|
|   15882.0|   17|
|   13030.0|   60|
|   17920.0|   95|
|   17682.0|   18|
|   18113.0|    1|
|   14867.0|    1|
+----------+-----+
only showing top 20 rows

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+-

In [42]:
from pyspark.sql.functions import count

retail_data_df\
    .groupBy("InvoiceNo")\
    .agg(count("Quantity").alias("quantity"),
         expr("count(Quantity)")
        )\
    .show()

+---------+--------+---------------+
|InvoiceNo|quantity|count(Quantity)|
+---------+--------+---------------+
|   537252|       1|              1|
|   537439|       1|              1|
|   537246|       7|              7|
|   537256|      11|             11|
|   537350|      12|             12|
|   537228|       1|              1|
|   537298|       9|              9|
|   537436|       1|              1|
|   537368|      15|             15|
|   537245|       3|              3|
|   537412|       1|              1|
|   537243|       2|              2|
|   537351|      32|             32|
|   537354|      38|             38|
|  C537373|       1|              1|
|   537435|      14|             14|
|  C537408|       2|              2|
|   537378|      20|             20|
|   537231|      18|             18|
|   537312|       9|              9|
+---------+--------+---------------+
only showing top 20 rows



In [52]:
retail_data_df\
    .groupBy("InvoiceNo")\
    .agg(expr("avg(Quantity)"),
         expr("mean(Quantity)"),
         expr("median(Quantity)"),
         expr("mode(Quantity)"),
         expr("stddev(Quantity)")
        )\
    .show()

+---------+------------------+------------------+----------------+--------------+------------------+
|InvoiceNo|     avg(Quantity)|    mean(Quantity)|median(Quantity)|mode(Quantity)|  stddev(Quantity)|
+---------+------------------+------------------+----------------+--------------+------------------+
|   536365| 5.714285714285714| 5.714285714285714|             6.0|             6| 1.799470821684875|
|   536366|               6.0|               6.0|             6.0|             6|               0.0|
|   536367| 6.916666666666667| 6.916666666666667|             5.0|             6|  8.09554684921771|
|   536368|              3.75|              3.75|             3.0|             3|               1.5|
|   536369|               3.0|               3.0|             3.0|             3|              null|
|   536370|             22.45|             22.45|            24.0|            24| 9.167878707749137|
|   536371|              80.0|              80.0|            80.0|            80|          

In [56]:
from pyspark.sql.functions import col, to_date
dfWithDate = retail_data_df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm:ss"))
dfWithDate.select("date").show()

+----------+
|      date|
+----------+
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
|2010-12-06|
+----------+
only showing top 20 rows

